In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...


C:\Users\jihyu\anaconda3\lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch: [ 0] [    0/  468] time: 0.7501, train_loss: 2.16120243, train_accuracy: 0.3906, test_Accuracy: 0.3030
Epoch: [ 0] [    1/  468] time: 0.9421, train_loss: 2.10570741, train_accuracy: 0.3281, test_Accuracy: 0.3316
Epoch: [ 0] [    2/  468] time: 1.1293, train_loss: 2.05928802, train_accuracy: 0.3828, test_Accuracy: 0.4163
Epoch: [ 0] [    3/  468] time: 1.3271, train_loss: 1.97322214, train_accuracy: 0.6172, test_Accuracy: 0.5998
Epoch: [ 0] [    4/  468] time: 1.4937, train_loss: 1.84255123, train_accuracy: 0.7344, test_Accuracy: 0.6866
Epoch: [ 0] [    5/  468] time: 1.6675, train_loss: 1.76345563, train_accuracy: 0.7578, test_Accuracy: 0.7295
Epoch: [ 0] [    6/  468] time: 1.8822, train_loss: 1.64766908, train_accuracy: 0.6953, test_Accuracy: 0.7176
Epoch: [ 0] [    7/  468] time: 2.0651, train_loss: 1.53603530, train_accuracy: 0.7031, test_Accuracy: 0.7077
Epoch: [ 0] [    8/  468] time: 2.2480, train_loss: 1.41808379, train_accuracy: 0.7031, test_Accuracy: 0.7069
Epoch: [ 0

Epoch: [ 0] [   75/  468] time: 14.8771, train_loss: 0.29039541, train_accuracy: 0.9141, test_Accuracy: 0.9125
Epoch: [ 0] [   76/  468] time: 15.0562, train_loss: 0.22436458, train_accuracy: 0.9141, test_Accuracy: 0.9102
Epoch: [ 0] [   77/  468] time: 15.2443, train_loss: 0.25563565, train_accuracy: 0.9141, test_Accuracy: 0.9115
Epoch: [ 0] [   78/  468] time: 15.4093, train_loss: 0.34875673, train_accuracy: 0.9141, test_Accuracy: 0.9120
Epoch: [ 0] [   79/  468] time: 15.5799, train_loss: 0.28163159, train_accuracy: 0.8906, test_Accuracy: 0.9121
Epoch: [ 0] [   80/  468] time: 15.7601, train_loss: 0.40230364, train_accuracy: 0.8828, test_Accuracy: 0.9100
Epoch: [ 0] [   81/  468] time: 15.9531, train_loss: 0.21964002, train_accuracy: 0.9609, test_Accuracy: 0.9092
Epoch: [ 0] [   82/  468] time: 16.1381, train_loss: 0.28058630, train_accuracy: 0.9062, test_Accuracy: 0.9076
Epoch: [ 0] [   83/  468] time: 16.3222, train_loss: 0.40761453, train_accuracy: 0.8984, test_Accuracy: 0.9088
E

Epoch: [ 0] [  150/  468] time: 28.6583, train_loss: 0.33641270, train_accuracy: 0.8984, test_Accuracy: 0.9267
Epoch: [ 0] [  151/  468] time: 28.8384, train_loss: 0.12589601, train_accuracy: 0.9688, test_Accuracy: 0.9258
Epoch: [ 0] [  152/  468] time: 29.0640, train_loss: 0.21814606, train_accuracy: 0.9688, test_Accuracy: 0.9295
Epoch: [ 0] [  153/  468] time: 29.2440, train_loss: 0.18132573, train_accuracy: 0.9531, test_Accuracy: 0.9321
Epoch: [ 0] [  154/  468] time: 29.4181, train_loss: 0.18597856, train_accuracy: 0.9609, test_Accuracy: 0.9310
Epoch: [ 0] [  155/  468] time: 29.6026, train_loss: 0.17571008, train_accuracy: 0.9531, test_Accuracy: 0.9298
Epoch: [ 0] [  156/  468] time: 29.8007, train_loss: 0.18419737, train_accuracy: 0.9531, test_Accuracy: 0.9286
Epoch: [ 0] [  157/  468] time: 29.9727, train_loss: 0.18189000, train_accuracy: 0.9453, test_Accuracy: 0.9279
Epoch: [ 0] [  158/  468] time: 30.1878, train_loss: 0.26881874, train_accuracy: 0.9141, test_Accuracy: 0.9286
E

Epoch: [ 0] [  224/  468] time: 44.7578, train_loss: 0.19688801, train_accuracy: 0.9297, test_Accuracy: 0.9304
Epoch: [ 0] [  225/  468] time: 45.0494, train_loss: 0.12678874, train_accuracy: 0.9844, test_Accuracy: 0.9312
Epoch: [ 0] [  226/  468] time: 45.3340, train_loss: 0.12036125, train_accuracy: 0.9688, test_Accuracy: 0.9334
Epoch: [ 0] [  227/  468] time: 45.5937, train_loss: 0.21604252, train_accuracy: 0.9375, test_Accuracy: 0.9344
Epoch: [ 0] [  228/  468] time: 45.8659, train_loss: 0.30967817, train_accuracy: 0.8906, test_Accuracy: 0.9379
Epoch: [ 0] [  229/  468] time: 46.1455, train_loss: 0.21368246, train_accuracy: 0.9297, test_Accuracy: 0.9389
Epoch: [ 0] [  230/  468] time: 46.4186, train_loss: 0.11481471, train_accuracy: 0.9766, test_Accuracy: 0.9390
Epoch: [ 0] [  231/  468] time: 46.6772, train_loss: 0.14069179, train_accuracy: 0.9453, test_Accuracy: 0.9388
Epoch: [ 0] [  232/  468] time: 46.9407, train_loss: 0.19807142, train_accuracy: 0.9297, test_Accuracy: 0.9383
E

Epoch: [ 0] [  298/  468] time: 59.6471, train_loss: 0.11176343, train_accuracy: 0.9609, test_Accuracy: 0.9524
Epoch: [ 0] [  299/  468] time: 59.8093, train_loss: 0.12231474, train_accuracy: 0.9609, test_Accuracy: 0.9529
Epoch: [ 0] [  300/  468] time: 59.9810, train_loss: 0.23692909, train_accuracy: 0.9531, test_Accuracy: 0.9536
Epoch: [ 0] [  301/  468] time: 60.1477, train_loss: 0.05801216, train_accuracy: 0.9922, test_Accuracy: 0.9529
Epoch: [ 0] [  302/  468] time: 60.3119, train_loss: 0.17011479, train_accuracy: 0.9609, test_Accuracy: 0.9525
Epoch: [ 0] [  303/  468] time: 60.4737, train_loss: 0.23616438, train_accuracy: 0.9375, test_Accuracy: 0.9519
Epoch: [ 0] [  304/  468] time: 60.6370, train_loss: 0.17549042, train_accuracy: 0.9688, test_Accuracy: 0.9504
Epoch: [ 0] [  305/  468] time: 60.8019, train_loss: 0.19384995, train_accuracy: 0.9297, test_Accuracy: 0.9499
Epoch: [ 0] [  306/  468] time: 60.9984, train_loss: 0.18157251, train_accuracy: 0.9453, test_Accuracy: 0.9503
E

Epoch: [ 0] [  372/  468] time: 72.2534, train_loss: 0.14136687, train_accuracy: 0.9531, test_Accuracy: 0.9526
Epoch: [ 0] [  373/  468] time: 72.4240, train_loss: 0.08587700, train_accuracy: 0.9844, test_Accuracy: 0.9531
Epoch: [ 0] [  374/  468] time: 72.5917, train_loss: 0.12098075, train_accuracy: 0.9766, test_Accuracy: 0.9537
Epoch: [ 0] [  375/  468] time: 72.7501, train_loss: 0.18164882, train_accuracy: 0.9375, test_Accuracy: 0.9554
Epoch: [ 0] [  376/  468] time: 72.9258, train_loss: 0.15085644, train_accuracy: 0.9531, test_Accuracy: 0.9566
Epoch: [ 0] [  377/  468] time: 73.1169, train_loss: 0.17240438, train_accuracy: 0.9531, test_Accuracy: 0.9574
Epoch: [ 0] [  378/  468] time: 73.2977, train_loss: 0.14008720, train_accuracy: 0.9609, test_Accuracy: 0.9571
Epoch: [ 0] [  379/  468] time: 73.4765, train_loss: 0.21802591, train_accuracy: 0.9453, test_Accuracy: 0.9570
Epoch: [ 0] [  380/  468] time: 73.6692, train_loss: 0.14080675, train_accuracy: 0.9688, test_Accuracy: 0.9562
E

Epoch: [ 0] [  446/  468] time: 84.9425, train_loss: 0.07765025, train_accuracy: 0.9844, test_Accuracy: 0.9594
Epoch: [ 0] [  447/  468] time: 85.1013, train_loss: 0.11499139, train_accuracy: 0.9766, test_Accuracy: 0.9598
Epoch: [ 0] [  448/  468] time: 85.2668, train_loss: 0.08781199, train_accuracy: 0.9688, test_Accuracy: 0.9607
Epoch: [ 0] [  449/  468] time: 85.4345, train_loss: 0.18570039, train_accuracy: 0.9375, test_Accuracy: 0.9617
Epoch: [ 0] [  450/  468] time: 85.6032, train_loss: 0.26378003, train_accuracy: 0.9062, test_Accuracy: 0.9611
Epoch: [ 0] [  451/  468] time: 85.7637, train_loss: 0.17529278, train_accuracy: 0.9609, test_Accuracy: 0.9593
Epoch: [ 0] [  452/  468] time: 85.9322, train_loss: 0.14455894, train_accuracy: 0.9609, test_Accuracy: 0.9583
Epoch: [ 0] [  453/  468] time: 86.0954, train_loss: 0.10845979, train_accuracy: 0.9766, test_Accuracy: 0.9552
Epoch: [ 0] [  454/  468] time: 86.2938, train_loss: 0.22118989, train_accuracy: 0.9219, test_Accuracy: 0.9554
E